In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import time
import itertools

datasetPath = "sampleSubmission.xls"
# Specify the delimiter as ',' and skip the header row
# Load the dataset with a numeric data type (e.g., float)
dataset = np.loadtxt(datasetPath, delimiter=",", skiprows=1, dtype=float)

k = 2
iterationCounter = 0
input = dataset

# ... (rest of the code remains the same)

In [ ]:
def plotClusterResult(listClusterMembers, centroid, iteration, converged):
    n = listClusterMembers.__len__()
    color = iter(cm.rainbow(np.linspace(0, 1, n)))
    plt.figure("result")
    plt.clf()
    plt.title("iteration-" + iteration)
    marker = itertools.cycle(('.', '*', '^', 'x', '+', 'o', 's'))
    for i in range(n):
        col = next(color)
        memberCluster = np.asmatrix(listClusterMembers[i])
        plt.scatter(np.ravel(memberCluster[:, 0]), np.ravel(memberCluster[:, 1]),
                             marker=marker.__next__(), s=100, c=col, label="klaster-"+str(i+1))
    for i in range(n):
        plt.scatter(centroid[i, 0], centroid[i, 1], marker=marker.__next__(),
                  s=200, c=col, label="centroid-" + str(i+1))
    if(converged == 0):
        plt.legend()
        plt.ion()
        plt.show()
        plt.pause(0.1)
    if(converged == 1):
        plt.legend()
        plt.show(block=True)

In [ ]:
def kMeans(data, centroidInit):
    nCluster = k  # banyaknya klaster
    global iterationCounter
    iterationCounter = 0
    centroidInit = np.matrix(centroidInit)
    # looping hingga konvergen
    while(True):
        iterationCounter += 1
        euclideanMatrixAllCluster = np.ndarray(shape=(data.shape[0], 0))
        # ulangi proses untuk semua klaster
        for i in range(0, nCluster):
            # The problem was in this line: centroidRepeated = np.repeat(centroidInit[:, i], data.shape[0], axis=0)
            # It was repeating the centroid values data.shape[0] times along axis=0, resulting in an incorrect shape.
            # The correct way is to repeat it data.shape[0] times as individual rows to match the shape of data
            # FIX: Use tile instead of repeat to create an array with the desired shape
            centroidRepeated = np.tile(centroidInit[i, :], (data.shape[0], 1))  # Corrected line

            deltaMatrix = abs(np.subtract(data, centroidRepeated))
            # hitung jarak Euclidean
            euclideanMatrix = np.sqrt(np.square(deltaMatrix).sum(axis=1))
            euclideanMatrixAllCluster = np.concatenate(
                (euclideanMatrixAllCluster, euclideanMatrix), axis=1
            )
        # tempatkan data ke klaster yang jarak Euclideannya paling dekat
        #The problem was in the next line, as clusterMatrix[i] was a single-element list
        #The fix is to just use clusterMatrix[i, 0]
        clusterMatrix = np.argmin(euclideanMatrixAllCluster, axis=1).astype(int)
        listClusterMember = [[] for i in range(k)]
        for i in range(0, data.shape[0]):
            # clusterMatrix[i] might be an array, convert it to an integer:
            listClusterMember[clusterMatrix[i,0]].append(data[i, :]) #Fixed This line!

        # hitung titik pusat klaster terbaru
        newCentroid = np.ndarray(shape=(0, centroidInit.shape[1]))
        for i in range(0, nCluster):
            #The following lines were not indented properly
            memberCluster = np.asarray(listClusterMember[i])
            centroidCluster = memberCluster.mean(axis=0)
            # Reshape centroidCluster to 2D if it's 1D
            if centroidCluster.ndim == 1:
                centroidCluster = centroidCluster.reshape(1, -1)
            newCentroid = np.concatenate((newCentroid, centroidCluster), axis=0)
        # hentikan loop jika sudah konvergen
        if((centroidInit == newCentroid).all()):
            break
        # update titik pusat klaster dengan nilai yang baru
        centroidInit = newCentroid
    # plot hasil klaster per iterasi
    plotClusterResult(listClusterMember, centroidInit, str(iterationCounter), 0) #Fixed indentation
    # diberi jeda 1 detik agar hasil plot klaster nyaman dilihat
    time.sleep(1)
    return listClusterMember, centroidInit

In [ ]:
centroidInit = initCentroid(input, k)
clusterResult, centroidResult = kMeans(input, centroidInit)
plotClusterResult(clusterResult, centroidResult, str(iterationCounter), + " (coverged)", 1)

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 1 dimension(s)